In [ ]:
from scipy.integrate import odeint
import time
import math
import numpy as np
import pylab as py
from math import *
import numpy as np
from scipy.optimize import newton
from matplotlib import animation, rc
from matplotlib import pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
from IPython.display import HTML
import os

In [ ]:
def numerical_solution(T, nt, nx, dt, dx, alpha):
    evo = []

    # Time-stepping loop
    for n in range(nt):
        T_new = T.copy()
        evo.append(T_new)
        for i in range(1, nx-1):
            T_new[i] = T[i] + (alpha * dt * (T[i+1] - 2*T[i] + T[i-1])) / dx**2
            
        # adiabatic Boundaries
        T_new[0] = T_new[1] # left border
        T_new[-1] = T_new[-2] # right border
        T = T_new

    diffusion_simulation = np.array(evo)
    
    return diffusion_simulation

In [ ]:
def Heat_eq_loss_1D(predicted_T, current_T, dx, dt, C):

    # Compute the 1D Laplacian (finite difference approximation)
    # laplacian = (current_T[2:] - 2*current_T[1:-1] + current_T[:-2]) / dx**2
    laplace = []
    xx_T = current_T*0
    for i in range(1, len(current_T)-1):
        xx_T[i] = (current_T[i+1] - 2*current_T[i] + current_T[i-1]) / dx**2
        laplace.append(xx_T)

    laplace_arr = np.array(laplace)

    # Compute the time derivative
    # time_derivative = (predicted_T[1:-1] - current_T[1:-1]) / dt
    time_derivative = (predicted_T - current_T) / dt
    # print(f"This is the ffirst nod {time_derivative[0]} ")
    # print(f"This is the derivative of the last nod {time_derivative[-1]}")
    loss_list = []

    for i in range(0, len(laplace[0])-2):
        physics_loss = time_derivative[i+1] - C * laplace[i]
        loss_list.append(physics_loss)

    return physics_loss

In [ ]:
    # PINN Information Conservation of Heat
    def Heat_conservation(predicted_Temp_dist, previous_Temp_dist, dx):
        
        energy_change = 0
        for i in range(len(previous_Temp_dist)-1):
            energy_change += (predicted_Temp_dist[i] - previous_Temp_dist[i]) * dx 
            
        return energy_change

In [ ]:
def create_inout_sequences(input_data, tw):
    inout_seq = []
    timesteps = len(input_data)
    for i in range(timesteps-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw:i+tw+1]
        inout_seq.append((train_seq ,train_label))
    return inout_seq

def preprocess_sol(train_data_normalized, train_window, nx):
    train_inout_seq= create_inout_sequences(train_data_normalized.reshape(len(train_data_normalized), nx), train_window)
    return train_inout_seq

In [ ]:
def data_splitter_and_scaler(scaler, Simulation_datasets, nx, test_size = 0.2):
    train_splits = []
    test_splits = []

    for dataset in Simulation_datasets:
        split_index = int(len(dataset) * (1 - test_size))
        train_split = dataset[:split_index]
        test_split = dataset[split_index:]
        train_splits.append(train_split)
        test_splits.append(test_split)
    
    for i in range(len(train_splits)):
        dataset = train_splits[i]
        train_window = 10
        
        train_data_normalized = scaler.fit_transform(dataset.reshape(-1, nx))
        train_data_normalized = torch.FloatTensor(train_data_normalized).reshape(len(dataset), nx)

        print(len(dataset))

        #A change of the reshape function inside the preprocess_sol might be required in case you change the amount of timesteps
        train_inout_seq = preprocess_sol(train_data_normalized, train_window, nx)
        
        train_splits[i] = train_inout_seq
        
    return train_splits, test_splits

In [ ]:
def multi_simulations(initial_condition_list, nt, nx, dt, dx, alpha):
    simulation_sets = []
    for initial_condition in initial_condition_list:
        diffusion_simulation = numerical_solution(initial_condition, nt, nx, dt, dx, alpha)
        # create a parameter list
        
        simulation_sets.append(diffusion_simulation)
        
    return simulation_sets

In [ ]:
def plot_the_temp_evolution(data, x, final_time, test_size = 0.2):
    # Parameter für den Plot
    t_test = final_time*test_size
    t = np.linspace(0, t_test, len(data))        # Zeitpunkte aus der Lösung
    T = data   # Temperaturwerte
    T = np.array(T).T
    Initial_temp_dist = T[:, 0]
    Final_temp_dist = T[:, -1]

    # Erstelle die Figur und die Achse
    fig = plt.figure(figsize=(10, 4))

    # 1. Subplot: 3D-Plot
    ax = fig.add_subplot(1, 2, 1, projection='3d')
    # Erstelle ein Gitter aus Position (x) und Zeit (t)
    X, T_grid = np.meshgrid(x, t)

    # Transponiere die Temperaturmatrix für den Plot (da X und T_grid vertauscht sind)
    Z = T.T

    # Erstelle die 3D-Oberfläche
    surf = ax.plot_surface(X, T_grid, Z, cmap='jet', edgecolor='none')
    ax.set_title("Plot der Temperaturverteilung über die Zeit")
    ax.set_xlabel("Position x (m)")
    ax.set_ylabel("Zeit t (s)")
    ax.set_zlabel("Temperatur T (°C)")
    ax.grid(True)

    # fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10, label="Temperatur", location="left")

    # 2. Subplot: 2D-Plot (Start- und Endzustand)
    ax2 = fig.add_subplot(1, 2, 2)
    ax2.plot(x, Initial_temp_dist, color='green', label="Anfangstemperatur")
    ax2.plot(x, Final_temp_dist, color='red', label="Endtemperatur")
    ax2.set_title("Anfang- und Endtemperatur")
    ax2.set_xlabel("Position x (m)")
    ax2.set_ylabel("Temperatur T (°C)")
    ax2.legend()
    plt.subplots_adjust(wspace=0.5)  # wspace steuert den horizontalen Abstand
    # plt.subplots_adjust(hspace=0.3)
    # Layout anpassen und anzeigen
    # plt.tight_layout()
    plt.show()

In [ ]:
def training_tester(model, data, num_layers, device, nx, hidden_layer_size = 100):    
    
    train_window = 10
    highest = len(data)- train_window
    
    # define test batch position
    call_begin_position = np.random.randint(len(data) - train_window - 1)
    call_end_position = call_begin_position + train_window
    
    scaler = MinMaxScaler(feature_range=(-1, 1))
    
    data_normalized = scaler.fit_transform(data.reshape(-1, nx))
    
    test_batch = data_normalized[call_begin_position:call_end_position].reshape(train_window,nx).tolist()
    test_batch = torch.tensor(test_batch).float()
    
    model.eval().cpu()
    
    model.c_h = (torch.zeros(num_layers, 1, hidden_layer_size).to(device),
                        torch.zeros(num_layers, 1, hidden_layer_size).to(device))
    prediction = model(test_batch).to(device)
    prediction = prediction.cpu().detach().numpy()
    
    test_comp_pos = call_end_position + 1
    test_data = data_normalized[test_comp_pos].reshape(1,nx).tolist()
    
    test_error = np.abs(test_data - prediction)
    test_error = np.mean(test_error)
    
    return prediction, test_data, test_error

In [ ]:
# Prediciton of the model
def OneStep(model, data, model_layers, nx, train_window = 10, steps = 100):
    print('data set length =', len(data))
    train_window = 10
    scaler = MinMaxScaler(feature_range=(-1, 1))
    train_data_normalized = scaler.fit_transform(data.reshape(-1, nx))
    fut_pred = len(data) -train_window
    test_inputs = train_data_normalized[0:train_window].reshape(train_window,nx).tolist()
    #print(test_inputs)
    s2 = train_data_normalized.reshape(len(data),nx).tolist()
    realdata = data
    model.eval()
    preds = test_inputs.copy()
    t2 = test_inputs

    hidden_layer_size = 100
    x = 0
    for i in range(fut_pred):
        seq = torch.FloatTensor(t2[i:])
        model.c_h = (torch.zeros(model_layers, 1, hidden_layer_size),
                        torch.zeros(model_layers, 1, hidden_layer_size))
        x = model(seq)
        preds.append(x.detach().numpy())
        t2.append(x.detach().numpy())
    actual_predictions = scaler.inverse_transform(np.array(preds ).reshape(-1,nx))

    pred_error = []
    mean_pred_error_of_line_list = []
    
    for i in range(len(data)):
        pred_error_line = []
        for j in range(len(data[i])):
            single_pred_error = np.abs(data[i][j] - actual_predictions[i][j])
            pred_error_line.append(single_pred_error)
        
        mean_error_line = np.mean(pred_error_line)
        mean_pred_error_of_line_list.append(mean_error_line)
        pred_error.append(pred_error_line)

    single_mean_error = np.mean(mean_pred_error_of_line_list)

    return actual_predictions, np.array(pred_error), single_mean_error

In [ ]:
def save_to_csv(data, actual_predictions, pred_error, filepath='.', filename='output.csv'):
    df_true_data = pd.DataFrame(data)
    df_actual_predictions = pd.DataFrame(actual_predictions)
    df_pred_error = pd.DataFrame(pred_error)
    
    df_combined = pd.concat([df_true_data, df_actual_predictions, df_pred_error], axis=1)
    
    full_path = os.path.join(filepath, f'pred_and_error_{filename}')
    df_combined.to_csv(full_path, index=False)


In [ ]:
def heat_distribution_and_error_plotter(actual_predictions, absolute_pred_error, L, nx, time):
    # create figure
    fig = plt.figure(figsize=(12, 5))
    
    # 1. Subplot: 3D-Plot with coloured error
    ax = fig.add_subplot(1, 2, 1, projection='3d')
    time = time[:len(actual_predictions)]
    # Create grid for position and time
    x = np.linspace(0, L, nx)
    X, T_grid = np.meshgrid(x, time)

    # Transform Temperature and Errox Matrix for the plot
    Z = actual_predictions
    error_z = absolute_pred_error

    # scale the error for the colouring
    error_normalized = (error_z - np.min(error_z)) / (np.max(error_z) - np.min(error_z))

    # Define colormap
    cmap = plt.colormaps['RdYlGn_r']

    # create the plot surface
    surf = ax.plot_surface(X, T_grid, Z, facecolors=cmap(error_normalized), edgecolor='none', shade=True)

    # In case you wanna plot the unscaled data
    # surf = ax.plot_surface(X, T_grid, Z, facecolors=cmap(absolute_pred_error), edgecolor='none', shade=True)

    # Create ScalarMappable-Object for colorbar
    mappable = plt.cm.ScalarMappable(cmap=cmap)
    mappable.set_array(absolute_pred_error)
    mappable.set_clim(vmin=np.min(absolute_pred_error), vmax=np.max(absolute_pred_error))

    # add colorbar
    cbar = fig.colorbar(mappable, ax=ax, shrink=0.5, aspect=10, location="left", label="Vorhersagefehler")

    ax.set_title("Temperaturverteilung über die Zeit mit Fehlerkennzeichnung")
    ax.set_xlabel("Position x (m)")
    ax.set_ylabel("Zeit t (s)")
    ax.set_zlabel("Temperatur T (°C)")

    # 2. Subplot: 2D-Plot begin temperature and end temperature
    ax2 = fig.add_subplot(1, 2, 2)
    Initial_temp_dist = actual_predictions[0]
    Final_temp_dist = actual_predictions[-1]

    ax2.plot(x, Initial_temp_dist, color='green', label="Anfangstemperatur")
    ax2.plot(x, Final_temp_dist, color='red', label="Endtemperatur")
    ax2.set_title("Anfangs- und Endtemperatur")
    ax2.set_xlabel("Position x (m)")
    ax2.set_ylabel("Temperatur T (°C)")
    ax2.legend()

    # more distance between plots
    plt.subplots_adjust(wspace=0.8)

    plt.show()

In [ ]:
def plot_temp_distribution(actual_predictions, L, nx, time):    
    # plot parameter
    x = np.linspace(0, L, nx) 
    t = time 
    T = actual_predictions
    #temp_dist beginning and end
    Initial_temp_dist = T[0]
    Final_temp_dist = T[-1]
    # create figure
    fig = plt.figure(figsize=(10, 4))
    t = time[:len(actual_predictions)]
    # 1. Subplot: 3D-Plot
    ax = fig.add_subplot(1, 2, 1, projection='3d')
    #Create meshgrit Position and time
    X, T_grid = np.meshgrid(x, t)

    # Tranponation of temperature matrix
    Z = T

    # create surface
    surf = ax.plot_surface(X, T_grid, Z, cmap='jet', edgecolor='none')
    ax.set_title("Plot der Temperaturverteilung über die Zeit")
    ax.set_xlabel("Position x (m)")
    ax.set_ylabel("Zeit t (s)")
    ax.set_zlabel("Temperatur T (°C)")
    # fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10, label="Temperatur")

    # 2. Subplot: 2D-Plot beginning and end
    ax2 = fig.add_subplot(1, 2, 2)
    ax2.plot(x, Initial_temp_dist, color='green', label="Anfangstemperatur")
    ax2.plot(x, Final_temp_dist, color='red', label="Endtemperatur")
    ax2.set_title("Anfangs- und Endtemperatur")
    ax2.set_xlabel("X")
    ax2.set_ylabel("Y")
    ax2.legend()

    plt.subplots_adjust(wspace=0.5)

    # fit layout
    plt.tight_layout()
    plt.show()